### **imports**

In [1]:
import pandas as pd
import numpy as np
from astropy.io import ascii
from astropy.table import QTable, Table, Column
from astropy import units as u
import string
import os
import upsilon

-------------------
| pyFFTW detected |
-------------------


### **making a table of all of the stars in the txt file**

In [2]:
column1 = []
column2 = []
column3 = []
column4 = []
column5 = []

with open('rrlyr_vsx_clean2.txt', 'r') as file:
    for line in file:
        components = line.split(' ', 4)
    
        try:
            magnitude = float(components[3])  # Try to convert the 4th component to float
        except ValueError:
            continue

        if magnitude < 15.5:
            column1.append(components[0])
            column2.append(float(components[1]))
            column3.append(float(components[2]))
            column4.append(magnitude)
            column5.append(components[4].strip())

final_table = QTable([column1, column2, column3, column4, column5], names = ("Star Type", "RA", "Dec", "Magnitude", "Other Info"))

In [3]:
final_table

Star Type,RA,Dec,Magnitude,Other Info
str24,float64,float64,float64,str29
RRAB,0.01521,35.36286,12.43,GM And
RRAB,0.01669,18.40698,15.171,CSS_J000004.0+182425
RRAB,0.03171,34.67397,14.95,SERIV 27
RRAB,0.06821,-72.77875,15.1,BP Tuc
RRAB,0.099,36.3287,14.46,NSVS 6313844
RRAB,0.1338,22.99388,14.671,CSS_J000032.1+225937
RRAB,0.13423,-54.29512,15.26,ASASSN-V J000032.21-541742.4
RRAB,0.14829,26.66375,12.96,GV Peg
RRC,0.15377,-60.98439,15.18,ASASSN-V J000036.92-605903.9


### **importing the asassn client**

In [4]:
from pyasassn.client import SkyPatrolClient	
client = SkyPatrolClient()

Welcome to ASAS-SN Skypatrol!

Current Deployment Version: 0.6.17 (26 JAN 2024)
Please upgrade your client if not up to date.



### **downloading a csv file with the data from the star at a specified index**

In [5]:
def classification_model(path, ra, dec, magnitude1, vsx_id, asassn_id):
  # Load a classification model.
  # (takes time, do only once if processing multiple lcs)
  rf_model = upsilon.load_rf_model()


  ############# Read input lc data #############
  # Read the input lightcurve file
  f = open(path, 'r')
  data = np.loadtxt(f, dtype={'names': ('JD', 'mag', 'err'),'formats': ('float64', 'float64', 'float64')})
  f.close()

  # Split it into three arrays
  date=data['JD']
  mag=data['mag']
  err=data['err']

  # Sigma-clip the input lightcurve
  date, mag, err = upsilon.utils.sigma_clipping(date, mag, err,  threshold=3, iteration=1)

  ############# Extract lightcurve features #############
  e_features = upsilon.ExtractFeatures(date, mag, err)
  e_features.run()
  features = e_features.get_features()

  ############# Classify #############
  # Classify the light curve
  label, probability, flag = upsilon.predict(rf_model, features)

  # Print results

  output_file = open(path.replace('.txt', '_classification.txt'), 'w')
  output_file.write("\n\n######### Classification results #########\n")
  output_file.write("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)\n")
  output_file.write("class = " + str(label) + "\n")
  output_file.write("class_probability = " + str(probability) + "\n")
  output_file.write("position = " + str(ra) + " " + str(dec) + "\n")
  output_file.write("magnitude = " + str(magnitude1) + "\n")
  output_file.write("vsx_id = " + str(vsx_id) + "\n")
  output_file.write("asassn_id = " + str(asassn_id) + "\n")
  output_file.close()

  print("\n\n######### Classification results #########\n")
  print("flag = " + str(flag) + " (0 - classification success, 1 - suspicious period)")
  print("class = " + str(label) )
  print("class_probability = " + str(probability) )
  print("\n\n######### Lightcurve features #########\n")

  for feature in features:
    print( str(feature) + " = " + str(features[feature]) )
  #  print feature," = ",features[feature]

In [6]:
folder_path = "/Users/zoesurles/Desktop/variable_star_classification/downloaded_lightcurves/"

In [7]:
num_iterations = 1
empty_list = []
for i in range(800, 850):
    if i == 724:
        continue

    try:
        search = client.cone_search(ra_deg = final_table["RA"][i], dec_deg = final_table["Dec"][i], radius=0.0003, catalog='aavsovsx', download=True)
    except:
        print("\n ################################################## error in index", i, "\n")
        continue

    savecsv = search.save(save_dir = folder_path, file_format="csv")[1]
    empty_list.append(savecsv)

    new_file = pd.read_csv(savecsv, skiprows=1)

    g_filter = new_file[new_file['phot_filter'] == 'g']
    quality = g_filter[g_filter['quality'] == 'G']
    web_format = quality[["jd", "mag", "mag_err"]]
    web_format.to_csv(savecsv.replace(".csv", ".txt"), sep='\t', index=False, header = False)

    classification_model(savecsv.replace(".csv", ".txt"), final_table["RA"][i], final_table["Dec"][i], final_table["Magnitude"][i], final_table["Other Info"][i], new_file["asas_sn_id"][0])

Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.98


######### Lightcurve features #########

amplitude = 0.4195754641296436
cusum = 0.26434629404437815
eta = 0.8080368933865423
hl_amp_ratio = 0.39643344212035897
kurtosis = -0.3434510245291955
n_points = 1982
period = 0.4912626242049114
period_SNR = 200.58826129627536
period_log10FAP = -253.84064419082267
period_uncertainty = 8.700457618948665e-05
phase_cusum = 0.3946787310321766
phase_eta = 0.3767782503131571
phi21 = -2.5569602252374355
phi31 = -1.7942135324159856
quartile31 = 0.5294317370056483
r21 = 0.35707913896007365
r31 = 0.15957119921476295
shapiro_w = 0.9457120895385742
skewness = -0.6614962162721678
slope_per10 = -0.007605159579080216
slope_per90 = 0.0067100363248882335
stetson_k = 0.6393716565069971
weighted_mean = 15.572821744696713
weighted_std = 0.389640615549808
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.95


######### Lightcurve features #########

amplitude = 0.23572918870625434
cusum = 0.07958266880717829
eta = 2.08958242939269
hl_amp_ratio = 0.47660970241518674
kurtosis = -0.807698626453329
n_points = 1963
period = 0.6507894693156572
period_SNR = 253.66912542677667
period_log10FAP = -308.2279233885296
period_uncertainty = 0.00017957768180681954
phase_cusum = 0.2603987106442053
phase_eta = 0.39364932941800057
phi21 = 3.8134765280312797
phi31 = 4.527382004749379
quartile31 = 0.32481628521988526
r21 = 0.28295756024367436
r31 = 0.17828294933249889
shapiro_w = 0.9261211156845093
skewness = -0.589293401376119
slope_per10 = -0.03231041223692725
slope_per90 = 0.027914561941224544
stetson_k = 0.7857892271983082
weighted_mean = 14.291885998094704
weighted_std = 0.18837032132050396
Pulled 1 of 1


...

/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_c
class_probability = 0.45


######### Lightcurve features #########

amplitude = 0.053895780279687354
cusum = 0.05322679052535829
eta = 1.51463076582308
hl_amp_ratio = 1.1734927267214252
kurtosis = -0.8699224439214923
n_points = 1817
period = 0.3587249080650652
period_SNR = 243.57432789236887
period_log10FAP = -286.9421836181096
period_uncertainty = 5.535308866466693e-05
phase_cusum = 0.22487105538770139
phase_eta = 0.596623722868163
phi21 = -0.1629808996162353
phi31 = -0.3269735835812462
quartile31 = 0.07329940751865394
r21 = 0.06339425636913477
r31 = 0.02303604240145164
shapiro_w = 0.9817374348640442
skewness = 0.11376837238323882
slope_per10 = -0.038650924895230795
slope_per90 = 0.04373413762221297
stetson_k = 0.8599548301279242
weighted_mean = 12.21112965679857
weighted_std = 0.041218020176908114
Pulled 1 of 1


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(




######### Classification results #########

flag = 0 (0 - classification success, 1 - suspicious period)
class = RRL_ab
class_probability = 0.99


######### Lightcurve features #########

amplitude = 0.32101130722627175
cusum = 0.18790064357294425
eta = 2.1476057205027117
hl_amp_ratio = 0.520270521980503
kurtosis = -1.044873676218128
n_points = 1551
period = 0.6814341006907031
period_SNR = 204.82250306120625
period_log10FAP = -227.4073829887248
period_uncertainty = 0.00016341346242559895
phase_cusum = 0.3734387291270108
phase_eta = 0.2081790919300727
phi21 = -2.158281174323504
phi31 = -4.111666710409966
quartile31 = 0.4632263983111784
r21 = 0.46865066120085075
r31 = 0.28766952190039335
shapiro_w = 0.932014524936676
skewness = -0.4474729312659965
slope_per10 = -0.03903587531158339
slope_per90 = 0.04174804757430999
stetson_k = 0.7649268529168203
weighted_mean = 14.51548098479968
weighted_std = 0.27378108613008284


/Users/zoesurles/miniconda3/lib/python3.11/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator RandomForestClassifier from version 0.24.1 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
